In [32]:
import os
import time
import numpy as np
import matplotlib.image as img 

In [33]:
N = len(next(os.walk('./data/'))[1])             #N is the number of test cases.
f_time = open('./output/time.txt', 'w')
f_results = open('./output/results.txt', 'w')

In [40]:
#program main loop
for i in range(N):
    path = "./data/"
    if i < 10:
        path = path + str(0) + str(i+1) + "/"
    else:
        path = path + str(i+1) + "/"
        
    print(str(0))
    w11 = img.imread(path + str(1) + "/1.png")
    w12 = img.imread(path + str(1) + "/2.png")
    w21 = img.imread(path + str(2) + "/1.png")
    w22 = img.imread(path + str(2) + "/2.png")
    w31 = img.imread(path + str(3) + "/1.png")
    w32 = img.imread(path + str(3) + "/2.png")
    test_im = img.imread(path + "test.png")
    
    start_time = time.time()
    #program logic here
      
    end_time = time.time()
    
    f_time.write(str(end_time - start_time) + "\n")
    
f_time.close()
f_results.close()

0


FileNotFoundError: [Errno 2] No such file or directory: './data/01/1/1.jpg'